In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import RetrievalMode, QdrantVectorStore
from qdrant_client.models import Filter, FieldCondition, MatchValue
import os
import re
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Verhindert inkonsistenten Fehler beim Erzeugen von Embeddings
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
encode_kwargs = {'prompt': 'query: '}

In [4]:
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large', encode_kwargs=encode_kwargs)

/Users/bastian/Desktop/BachelorProjektAbgabe/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:datasets:PyTorch version 2.6.0 available.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-large


In [5]:
token_collection = "intfloat_multilingual-e5-large_token_based_chunks"
qdrant_url = "http://localhost:6333"
token_client = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    prefer_grpc=False,
    collection_name=token_collection,
    url=qdrant_url,
)

INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks "HTTP/1.1 200 OK"


In [6]:
recursive_collection = "intfloat_multilingual-e5-large_recursive_chunks"
recursive_client = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    prefer_grpc=False,
    collection_name=recursive_collection,
    url=qdrant_url,
)

INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks "HTTP/1.1 200 OK"


In [7]:
fragen = {
    1: "Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben?",
    7: "Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen?",
    10: "Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen?"
}

In [33]:
result = token_client.similarity_search_with_score(query=fragen[1], k=5)

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


In [34]:
result[0]

(Document(metadata={'id': 1021, 'studiengang': 'Elektrotechnik', 'abschluss': 'Master', 'standort': 'Meschede', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/elektrotechnik_m_eng_/index.php', 'hash': '23d5055fb6686b7f964ad1114b3e41d5d939a5dfe2428fa974f4bd6af87e72a5', '_id': '579aeac7-499f-4213-b1ea-0e9425bb6c64', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='Jahre abhängig von den Studienvoraussetzungen) ### Studienort Meschede ### Beiträge und Gebühren ### Hauptunterrichtssprache Deutsch (für internationale Bewerbende: Level DSH3 / TestDaf5 / C2) ## Vor Beginn des Elektrotechnik Master-Studiums – Voraussetzungen &amp; Bewerbungsverfahren ### Voraussetzungen ##### Für den viersemestrigen Masterstudiengang - Abschluss eines sechssemestrigen Studiums mit mind. 180 Credits in einem der unten aufgeführten Studiengänge mit einer Gesamtnote von mindestens 2,7 (oder mit einer Gesamtnote von mindestens 3,0 und einer Bachelorarbeit mit 

In [35]:
first_page_content = result[0][0].page_content

In [36]:
first_page_content

'Jahre abhängig von den Studienvoraussetzungen) ### Studienort Meschede ### Beiträge und Gebühren ### Hauptunterrichtssprache Deutsch (für internationale Bewerbende: Level DSH3 / TestDaf5 / C2) ## Vor Beginn des Elektrotechnik Master-Studiums – Voraussetzungen &amp; Bewerbungsverfahren ### Voraussetzungen ##### Für den viersemestrigen Masterstudiengang - Abschluss eines sechssemestrigen Studiums mit mind. 180 Credits in einem der unten aufgeführten Studiengänge mit einer Gesamtnote von mindestens 2,7 (oder mit einer Gesamtnote von mindestens 3,0 und einer Bachelorarbeit mit einer besseren Note als 2,3) ##### Für den dreisemestrigen Masterstudiengang - Abschluss eines siebensemestrigen Studiums mit einem Umfang von mind. 210 Credits in einem der unten auf geführten Studiengänge mit einer Gesamtnote von mindestens 2,7 (oder mit einer Gesamtnote von mindestens 3,0 und einer Bachelorarbeit mit einer besseren Note als 2,3) ##### Anerkannte Bachelor- oder Diplomstudiengänge - Elektrotechnik 

In [37]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_name_or_path="intfloat/multilingual-e5-large")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-large


In [38]:
# Embedding der Frage
query_embedding = model.encode(f"query: {fragen[1]}")

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.55it/s]


In [39]:
# Embedding des ersten Dokuments
document_embedding = model.encode(f"passage: {first_page_content}")

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.40it/s]


In [40]:
query_embedding

array([ 0.01067872,  0.00546123,  0.00756248, ..., -0.02824133,
       -0.04448807,  0.01577423], dtype=float32)

In [41]:
document_embedding

array([-0.00250476, -0.00498887,  0.00266136, ..., -0.02109128,
       -0.04612838,  0.02727768], dtype=float32)

In [48]:
from sentence_transformers.util import cos_sim, pairwise_cos_sim, dot_score, euclidean_sim,pytorch_cos_sim
similarity = cos_sim(query_embedding, document_embedding)

In [43]:
similarity

tensor([[0.8931]])

In [44]:
dot_score(query_embedding, document_embedding)

tensor([[0.8931]])

In [47]:
euclidean_sim(query_embedding, document_embedding)

tensor([[-0.4623]])

In [49]:
pytorch_cos_sim(query_embedding, document_embedding)

tensor([[0.8931]])

With Score ist also die Kosinus-Ähnlichkeit, die dabei zurückgegeben wird.
Dann kann so der Score auch ermittelt werden, wenn der SelfQueryRetriever keine Scores zurückgibt.

Also Embeddings für die Fragen erstellen, Dokumente abrufen, page_content Embedding des jeweiligen Dokumentes erstellen und dann die Kosinus-Ähnlichkeit zwischen Frage und page_content berechnen und den Score speichern.

Einfachere Möglichkeit ist:
https://python.langchain.com/docs/how_to/add_scores_retriever/#selfqueryretriever

Die Methode des SelfQueryRetriever kann überschrieben werden, um die Scores bei der Abfrage mit zurückzugeben.


In [2]:
from ranx import Qrels, Run, evaluate, compare

In [20]:
qrels_dict = {
    "q_1": {
        "d_1021": 1,
        "d_1020": 0.2,
    }
}

run_dict = {
    "q_1": {
        "d_1021": 0.94,
        "d_1020": 0.94,
        "d_787": 0.94,
        "d_788": 0.94,
        "d_1024": 0.94,
    }
}

qrels = Qrels(qrels_dict)
run = Run(run_dict)

In [21]:
score = evaluate(qrels, run, "ndcg@5")
print(score)

0.6309297535714575


In [22]:
score_dict = evaluate(qrels, run, ["ndcg@3", "map@5", "mrr","recall"])
print(score_dict)

{'ndcg@3': np.float64(0.6309297535714575), 'map@5': np.float64(0.5), 'mrr': np.float64(0.5), 'recall': np.float64(1.0)}


Die Metriken sind allgemein nicht im Zusammenhang mit RAG erst entstanden.
Sie gehören zum Bereich des Information Retrievals, eignen sich daher zur Evaluation des Retrievals, um die Performance zwischen den Embedding-Modellen und Collections zu vergleichen.